# Landing Page Scoring Prototype (FastAPI + OpenAI)

This notebook launches a **FastAPI** server that scores landing pages using an **LLM**.

- `POST /score`: analyze one page (clarity, credibility, CTA)
- `POST /compare`: compare **before** vs **after**

We defined the main function, `llm_score_page`, in such a way it will auto-fallback to mock when needed, i.e.
- for the purpose of the demo, to avoid having to provide a key, from a paid API ;
- if the LLLM API is failing to answer the requests

The mock, `_mock_score`, has been designed very simple and heuristics so the demo yields sensible, deterministic output.

Parameters: in the section **2) Set your OpenAI API key** of this notebook, if you have an Openai API key, you can set the environment variable `OPENAI_API_KEY` and `USE_MOCK` to `"0"`.
Otherwise you can just set the environment variable `USE_MOCK` to `"1"`.

other wise just leave it .

> Tip: Run cells top-to-bottom. The server starts in the background so you can test directly from the notebook.


## 1) Install dependencies

In [ ]:
# If running locally, uncomment to install
!pip install -q fastapi uvicorn pydantic openai>=1.0.0 nest_asyncio requests

## 2) Set your OpenAI API key

In [ ]:
import os

os.environ["USE_MOCK"] = "0"

print("USE_MOCK environment variable's value is", os.environ.get("USE_MOCK"))

if os.getenv("USE_MOCK") == "0" :

    os.environ["OPENAI_API_KEY"] = "sk-..."  # Uncomment and paste your key

    if not os.environ.get("OPENAI_API_KEY") :
          print("⚠️ Mock mode is OFF, OPENAI_API_KEY is not set. Set it via environment or in this cell before starting the server.")
    else :
          print("⚠️ Mock mode is OFF, the demo uses the API key provided.")

else :
      print("Mock mode is ON, if you don't have an API key we propose a simple and heuristic scoring")


os.environ["PORT"] = "8000"



USE_MOCK environment variable's value is 0
⚠️ Mock mode is OFF, the demo uses the API key provided.


## 3) Define FastAPI app and LLM helper

In [ ]:
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field
from fastapi import FastAPI
from openai import OpenAI
import json, re
import requests, json


# Main

def _mock_score(html: str, criteria: List[str]) -> Dict[str, Any]:
    # Very simple heuristics so the demo yields sensible, deterministic output
    txt = re.sub(r"<[^>]+>", " ", html or "").lower()
    have_h1   = bool(re.search(r"<h1[^>]*>.*?</h1>", html or "", flags=re.I|re.S))
    have_title= bool(re.search(r"<title[^>]*>.*?</title>", html or "", flags=re.I|re.S))
    have_btn  = ("<button" in (html or "").lower()) or any(k in txt for k in ["get started","start free","sign up","contact","try free"])
    trust_kw  = sum(k in txt for k in ["trusted by","testimonials","review","reviews","privacy","secure","https","iso","gdpr","clients","partners"])
    words     = len(txt.split())

    scores: Dict[str, Dict[str, Any]] = {}

    if "clarity" in criteria:
        s = 4 + (3 if have_h1 else 0) + (2 if have_title else 0) + (1 if 50 <= words <= 400 else 0)
        s = max(1, min(10, s))
        scores["clarity"] = {"score": s, "feedback": ("Clear headline." if have_h1 else "Add a clear H1.") + (" Keep copy concise." if words>400 else "")}

    if "credibility" in criteria:
        s = 3 + min(7, trust_kw)  # up to +7 from trust signals
        s = max(1, min(10, s))
        fb_parts = []
        if trust_kw == 0: fb_parts.append("Add testimonials/trust badges.")
        if "https" not in txt: fb_parts.append("Show security/privacy info.")
        scores["credibility"] = {"score": s, "feedback": " ".join(fb_parts) or "Good trust signals."}

    if "cta" in criteria:
        s = 4 + (4 if have_btn else 0) + (2 if have_h1 else 0)
        s = max(1, min(10, s))
        scores["cta"] = {"score": s, "feedback": ("CTA visible." if have_btn else "Add a prominent CTA.")}

    # Fill missing criteria if any
    for c in criteria:
        scores.setdefault(c, {"score": 5, "feedback": "OK"})

    overall = round(sum(v["score"] for v in scores.values())/len(scores), 2)
    return {"scores": scores, "overall": overall, "notes": "Mock mode: heuristic scoring"}

# llm_score_page defined to auto-fallback to mock when needed
def llm_score_page(html: str, url: Optional[str], criteria: Optional[List[str]] = None, model: str = "gpt-5-mini") -> Dict[str, Any]:
    from openai import OpenAI
    from openai import RateLimitError, AuthenticationError, APIStatusError  # available in new SDKs; if missing, we’ll catch generic Exception
    client = OpenAI()

    criteria = criteria or ["clarity","credibility","cta"]

    # Force mock if requested or no key present
    if USE_MOCK or not os.getenv("OPENAI_API_KEY"):
        return _mock_score(html, criteria)

    # Build messages
    SYSTEM_PROMPT = (
        "You are an assistant that evaluates marketing landing pages.\n"
        "Score each requested criterion from 1 to 10 and give concise, actionable feedback (max ~25 words each).\n"
        "Be consistent and fair across pages. When content is missing, explain briefly.\n"
        "Return ONLY valid JSON. Use integers for scores."
    )
    crit_list = "\n".join(f"- {c}" for c in criteria)
    user_prompt = f"""Evaluate the landing page below.
URL: {url or "N/A"}

Criteria:
{crit_list}

Return a JSON object with:
{{
  "scores": {{
    "<criterion>": {{ "score": <int 1-10>, "feedback": "<string>" }}
  }},
  "overall": <float>,
  "notes": "<short rationale>"
}}

Page HTML (truncated OK):
<<<HTML_START>>>
{html}
<<<HTML_END>>>"""

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt},
    ]

    # Try OpenAI; if quota/auth fails, fall back to mock (but still return 200 JSON)
    try:
        resp = client.chat.completions.create(
            model=model, temperature=0.2, messages=messages,
            response_format={"type": "json_object"}
        )
        content = resp.choices[0].message.content
        data = json.loads(content)
        # Fill any missing parts
        data.setdefault("scores", {})
        for c in criteria:
            data["scores"].setdefault(c, {"score": 0, "feedback": ""})
        if "overall" not in data or not isinstance(data["overall"], (int,float)):
            vals = [v["score"] for v in data["scores"].values()]
            data["overall"] = round(sum(vals)/len(vals), 2) if vals else 0.0
        data.setdefault("notes", "")
        return data

    except (RateLimitError, AuthenticationError, APIStatusError) as e:
        # Quota/auth/server issue → degrade gracefully
        mock = _mock_score(html, criteria)
        mock["notes"] = f"Mock fallback due to LLM error: {type(e).__name__}"
        return mock

    except Exception as e:
        # Any other unexpected error → degrade gracefully
        mock = _mock_score(html, criteria)
        mock["notes"] = f"Mock fallback due to error: {e.__class__.__name__}"
        return mock

'''
def normalize_output(raw_text: str, criteria: List[str]) -> Dict[str, Any]:
    try:
        data = json.loads(raw_text)
    except Exception:
        import re
        m = re.search(r"\{[\s\S]*\}$", raw_text.strip())
        if not m:
            raise
        data = json.loads(m.group(0))

    data.setdefault("scores", {})
    for c in criteria:
        data["scores"].setdefault(c, {"score": 0, "feedback": ""})

    vals = [v.get("score") for v in data["scores"].values() if isinstance(v.get("score"), (int, float))]
    data["overall"] = round(sum(vals)/len(vals), 2) if vals else 0.0
    data.setdefault("notes", "")
    return data

def llm_score_page(html: str, url: Optional[str], criteria: Optional[List[str]] = None, model: str = "gpt-5-mini") -> Dict[str, Any]:
    criteria = criteria or DEFAULT_CRITERIA
    messages = build_messages(html, url, criteria)
    try:
        resp = client.chat.completions.create(
            model=model,
            temperature=0.2,
            messages=messages,
            response_format={ "type": "json_object" }
        )
        content = resp.choices[0].message.content
    except Exception:
        resp = client.chat.completions.create(
            model=model,
            temperature=0.2,
            messages=messages
        )
        content = resp.choices[0].message.content

    return normalize_output(content, criteria)

'''

class ScoreRequest(BaseModel):
    html: str = Field(..., description="Raw HTML of the page")
    url: Optional[str] = Field(None, description="Optional URL (for context)")
    criteria: Optional[List[str]] = Field(None, description="List of criteria to evaluate")

class ScoreResponse(BaseModel):
    scores: Dict[str, Dict[str, Any]]
    overall: float
    notes: str

class CompareRequest(BaseModel):
    before_html: str
    after_html: str
    url: Optional[str] = None
    criteria: Optional[List[str]] = None

class CompareResponse(BaseModel):
    before: ScoreResponse
    after: ScoreResponse
    delta: Dict[str, float]

app = FastAPI(title="Landing Page Scoring Prototype - API", version="0.1.0")

@app.get("/healthz")
def healthz():
    return {"status": "ok"}

@app.post("/score", response_model=ScoreResponse)
def score_page(req: ScoreRequest):
    result = llm_score_page(req.html, req.url, req.criteria)
    return result

@app.post("/compare", response_model=CompareResponse)
def compare_pages(req: CompareRequest):
    criteria = req.criteria or ["clarity", "credibility", "cta"]
    before = llm_score_page(req.before_html, req.url, criteria)
    after = llm_score_page(req.after_html, req.url, criteria)
    delta = {}
    for c in criteria + ["overall"]:
        b = before["scores"][c]["score"] if c in before["scores"] else before["overall"]
        a = after["scores"][c]["score"] if c in after["scores"] else after["overall"]
        delta[c] = round((a - b), 2)
    return {"before": before, "after": after, "delta": delta}


<>:128: SyntaxWarning: invalid escape sequence '\{'
<>:128: SyntaxWarning: invalid escape sequence '\{'
/tmp/ipython-input-1562034320.py:128: SyntaxWarning: invalid escape sequence '\{'
  m = re.search(r"\{[\s\S]*\}$", raw_text.strip())


## 4) Start FastAPI (background)

In [ ]:
import nest_asyncio, threading, uvicorn, socket
nest_asyncio.apply()

def find_free_port(start=8000, end=8100):
    import socket
    for port in range(start, end+1):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(("127.0.0.1", port))
                return port
            except OSError:
                continue
    raise RuntimeError("No free port found in range.")

PORT = find_free_port()

def run_server():
    uvicorn.run(app, host="127.0.0.1", port=PORT, log_level="info")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()
print(f"🚀 API running at http://127.0.0.1:{PORT}")

🚀 API running at http://127.0.0.1:8002


INFO:     Started server process [1164]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


## 5) Test call to `/score`

In [ ]:
import requests, json

sample_html = """
<html>
  <head><title>Grow your leads fast</title></head>
  <body>
    <h1>Boost your lead generation</h1>
    <p>Join 1,200+ businesses using our platform.</p>
    <button>Get started</button>
    <footer>No testimonials yet.</footer>
  </body>
</html>
"""

# Better handling errors - Make the client test more robust (avoid JSONDecodeError)
resp = requests.post(f"http://127.0.0.1:{PORT}/score", json={
    "html": sample_html,
    "url": "https://example.com",
    "criteria": ["clarity", "credibility", "cta"]
})

print(resp.status_code, resp.headers.get("content-type"))
if "application/json" in (resp.headers.get("content-type") or ""):
    print(json.dumps(resp.json(), indent=2))
else:
    print(resp.text[:1000])  # show text body if not JSON


INFO:     127.0.0.1:49634 - "POST /score HTTP/1.1" 200 OK
200 application/json
{
  "scores": {
    "clarity": {
      "score": 9,
      "feedback": "Clear headline."
    },
    "credibility": {
      "score": 4,
      "feedback": "Show security/privacy info."
    },
    "cta": {
      "score": 10,
      "feedback": "CTA visible."
    }
  },
  "overall": 7.67,
  "notes": "Mock fallback due to LLM error: RateLimitError"
}


## 6) Test call to `/compare`

In [ ]:
import requests, json
before_html = """
<html>
  <body>
    <h1>Boost your lead generation</h1>
    <p>We help teams grow.</p>
    <button>Learn more</button>
  </body>
</html>
"""

after_html = """
<html>
  <body>
    <h1>Boost your lead generation</h1>
    <p>Join 1,200+ businesses using our platform. 30-day free trial.</p>
    <button>Start free trial</button>
    <section><h2>Trusted by</h2><ul><li>ACME</li><li>Globex</li></ul></section>
  </body>
</html>
"""

resp = requests.post(f"http://127.0.0.1:{PORT}/compare", json={
    "before_html": before_html,
    "after_html": after_html,
    "url": "https://example.com",
    "criteria": ["clarity", "credibility", "cta"]
})
print(resp.status_code)
print(json.dumps(resp.json(), indent=2))

INFO:     127.0.0.1:33656 - "POST /compare HTTP/1.1" 200 OK
200
{
  "before": {
    "scores": {
      "clarity": {
        "score": 7,
        "feedback": "Clear headline."
      },
      "credibility": {
        "score": 3,
        "feedback": "Add testimonials/trust badges. Show security/privacy info."
      },
      "cta": {
        "score": 10,
        "feedback": "CTA visible."
      }
    },
    "overall": 6.67,
    "notes": "Mock fallback due to LLM error: RateLimitError"
  },
  "after": {
    "scores": {
      "clarity": {
        "score": 7,
        "feedback": "Clear headline."
      },
      "credibility": {
        "score": 4,
        "feedback": "Show security/privacy info."
      },
      "cta": {
        "score": 10,
        "feedback": "CTA visible."
      }
    },
    "overall": 7.0,
    "notes": "Mock fallback due to LLM error: RateLimitError"
  },
  "delta": {
    "clarity": 0.0,
    "credibility": 1.0,
    "cta": 0.0,
    "overall": 0.33
  }
}
